In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
import os
import urllib.request #import urllib works for python 2

import tensorflow as tf
import numpy as np

BOSTON_TRAINING = "boston_train.csv"
BOSTON_TRAINING_URL = "http://download.tensorflow.org/data/boston_train.csv"

BOSTON_TEST = "boston_test.csv"
BOSTON_TEST_URL = "http://download.tensorflow.org/data/boston_test.csv"

BOSTON_PREDICT = "boston_predict.csv"
BOSTON_PREDICT_URL = "http://download.tensorflow.org/data/boston_predict.csv"

In [5]:
if not os.path.exists(BOSTON_TRAINING):
  raw = urllib.request.urlopen(BOSTON_TRAINING_URL).read()
  with open(BOSTON_TRAINING,'w') as f:
    f.write(raw.decode()) 

if not os.path.exists(BOSTON_TEST):
  raw = urllib.request.urlopen(BOSTON_TEST_URL).read()
  with open(BOSTON_TEST,'w') as f:
    f.write(raw.decode())
    
if not os.path.exists(BOSTON_PREDICT):
  raw = urllib.request.urlopen(BOSTON_PREDICT_URL).read()
  with open(BOSTON_PREDICT,'w') as f:
    f.write(raw.decode())
    
    

In [6]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [7]:
feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES]

In [8]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                          hidden_units=[10, 10],
                                          model_dir="/tmp/boston_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11fec7780>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}


In [9]:
def input_fn(data_set):
  feature_cols = {k: tf.constant(data_set[k].values)
                  for k in FEATURES}
  labels = tf.constant(data_set[LABEL].values)
  return feature_cols, labels

In [10]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=5000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/boston_model/model.ckpt.
INFO:tensorflow:loss = 293.651, step = 1
INFO:tensorflow:global_step/sec: 763.703
INFO:tensorflow:loss = 92.4398, step = 101 (0.132 sec)
INFO:tensorflow:global_step/sec: 709.039
INFO:tensorflow:loss = 82.5393, step = 201 (0.141 sec)
INFO:tensorflow:global_step/sec: 825.431
INFO:tensorflow:loss = 77.4206, step = 301 (0.121 sec)
INFO:tensorflow:global_step/sec: 649.819
INFO:tensorflow:loss = 73.5605, step = 401 (0.154 sec)
INFO:tensorflow:global_step/sec: 625.997
INFO:tensorflow:loss = 70.9561, step = 501 (0.160 sec)
INFO:tensorflow:global_step/se

DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x10e24cd68>, 'hidden_units': [10, 10], 'feature_columns': (_RealValuedColumn(column_name='crim', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='zn', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='indus', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='nox', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='rm', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='dis', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='tax', dimension=1, default_value=None, dtype=tf.float32,

In [11]:
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-08-03-19:33:38
INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-08-03-19:33:39
INFO:tensorflow:Saving dict for global step 5000: global_step = 5000, loss = 12.1565


In [12]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 12.156476


In [14]:
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
# .predict() returns an iterator; convert to a list and print predictions
predictions = list(itertools.islice(y, 6))
print ("Predictions: {}".format(str(predictions)))

Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
Predictions: [34.596008, 19.897644, 22.6633, 36.555283, 14.575801, 18.593182]
